# Heuristic Algorithm Experiment

In this experiment we evaluate whether the coordinates we have annotated via graph layouting are usable to inform heuristic search algorithms.

The goals of this experiment are twofold:
- Identify, which heuristic algorithm performs best with the annotated coordinates.
- Identify, which method of coordinate annotation works best.

The notebook contains multiple experiments with the following purposes:
- Experiment 1: Identify good scale factors for the generated coordinates
- Experiment 2: Test which graph layout algorithm works best.
- Experiment 3: Test all informed search algorithms on the best layout algorithm.

In [ ]:
# correct working directory.
# This is necessary for imports because the notebook is not in the main folder of the project. 
if not "working_directory_corrected" in vars():
    %cd ..
    working_directory_corrected = True


import pandas as pd

from evaluation.timed_experiment import Timed_Experiment
from algorithms.dijkstra import Dijkstra
from algorithms.a_star import A_Star
from algorithms.heuristic_search import Heuristic_Search


# load dataset
from data.dataset import Dataset

## Dataset
This experiment is based on data generated with the script *generate_coordinates.py*. This script applies all algorithms identified in *coordinate_annotation.ipynb*..While executing this script, we had to exclude some of the algorithms due to the following reasons:
- kamada_kawai and mds ran into memory issues and could not be executed
- davidson_harel and graphopt have not delivered a result within an hour of time. To keep the experiment manageable time-wise, they have been excluded based on this.

For the remaining algorithms we collected five different coordinates each.



## Experiment 1 - Scale factor

As discussed in coordinate_annotation.ipynb, some coordinates where extremely close together. To give A* a fair chance we would like to rectify this by scaling coordinates so that they are in similar maximal distances.

### Procedure

The cell below loads the first coordinate set of each algorithm and prints the minimum and maximum x and y coordinates.
Scale factors will be determined as potenties of 10 such that both coordinates are at least 10000 apart.

In [ ]:
from asyncio.windows_events import INFINITE


for algorithm_name in ["auto", "drl", "fruchterman_reingold"]:
    file_name = f"if_{algorithm_name}_1"

    # load and convert graph
    dataset = Dataset()
    graph = dataset.load_graph()
    dataset.convert_to_spatial(graph,file_name)

    # find min and max of coordinates
    min_x = min(graph.node_positions, key= lambda coord: coord[0])[0]
    max_x = max(graph.node_positions, key= lambda coord: coord[0])[0]
    min_y = min(graph.node_positions, key= lambda coord: coord[1])[1]
    max_y = max(graph.node_positions, key= lambda coord: coord[1])[1]

    print("\n" + algorithm_name)
    print(f"min_x: {min_x}, max_x: {max_x}, Distance: {max_x - min_x}")		
    print(f"min_y: {min_y}, max_y: {max_y}, Distance: {max_y - min_y}")	

    

### Results:
The following distances have been calculated:

| Algorithm | X distance | Y Distance |
| --- | --- | --- |
| auto | 1954 | 1906 |
| drl | 1922 | 1941 |
| fruchterman_reingold | 549 | 558 |

### Conclusions:

We will apply the following scale factors:
- auto: 10
- drl: 10
- fruchterman_reingold: 100


## Experiment 2: Which coordinates are best?

The goal of this experiment is to determine which of the coordinates produced by the layout algorithms perform best.

### Procedure

In the below cell we test all heuristic search methods with all coordinates.

We will test Dijkstra's algorithm, A* and Heuristic search - all using manhattan distance as heuristic. Manhattan distance has been used due to it's lower computational cost, compared to euclidean distance.

Each algorithm will be tested on all 5 variants of the three layout algorithms. To have a comparison benchmark we also run them without coordinates. 
Each run will be comprised of 100 planning problems, all with the same seed. 

From each experiment we will collect the following data:
- *average_time*: The average time required to solve one problem.
- *nr_extended*: The number of nodes that were extended.
- *time per extension*: The average time required for one extension.

We will select the best layout algorithm and version based on the average time.

In [4]:

algorithm_names = [None, "auto", "drl", "fruchterman_reingold"]
scale_factors = [0, 10,10,100]
planners = [Dijkstra, A_Star, Heuristic_Search] 

for index in range(len(algorithm_names)):
    algorithm_name = algorithm_names[index]	
    scale_factor = scale_factors[index]	
    
    for variation in range(5):
        file_name = f"if_{algorithm_name}_{variation}"
        dataset = Dataset()
        graph = dataset.load_graph()
        if algorithm_name is not None:
            dataset.convert_to_spatial(graph,file_name, scale_factor=scale_factor)

        for planner in planners:

            print(f"\nrunning {planner} on algorithm {algorithm_name} with variation {variation}")
            experiment = Timed_Experiment(graph, planner, 100, random_seed=42, verbose = False)
            experiment.run()
            print("Average Time: ", int(experiment.get_average_time()), "ns")
            print("Nr Extended: ", experiment.get_nr_extensions())
            print("Time per extension: ", int(experiment.get_average_extension_time()), "ns")



running <class 'algorithms.dijkstra.Dijkstra'> on algorithm None with variation 0
Average Time:  165020572 ns
Nr Extended:  3388818
Time per extension:  4869 ns

running <class 'algorithms.a_star.A_Star'> on algorithm None with variation 0
Average Time:  181506869 ns
Nr Extended:  3388818
Time per extension:  5356 ns

running <class 'algorithms.heuristic_search.Heuristic_Search'> on algorithm None with variation 0
Average Time:  116332918 ns
Nr Extended:  3290487
Time per extension:  3535 ns

running <class 'algorithms.dijkstra.Dijkstra'> on algorithm None with variation 1
Average Time:  165797787 ns
Nr Extended:  3388818
Time per extension:  4892 ns

running <class 'algorithms.a_star.A_Star'> on algorithm None with variation 1
Average Time:  183288321 ns
Nr Extended:  3388818
Time per extension:  5408 ns

running <class 'algorithms.heuristic_search.Heuristic_Search'> on algorithm None with variation 1
Average Time:  118698189 ns
Nr Extended:  3290487
Time per extension:  3607 ns

run

### Results

The following table collects the average time over all experiments. Since the runs without layout algorithms are identical we will only pick one representation from them.

| Experiment | Dijkstra | A* | Heuristic Search |
| --- | --- | --- | --- |
| Benchmark 4| 150 ms | 173 ms | **109 ms**|
| auto 0 | 119 ms | 137 ms | 137 ms | 
| auto 1 | 120 ms | 140 ms | 142 ms |
| auto 2 | 118 ms | 138 ms | 137 ms |
| auto 3 | 122 ms | 139 ms | 140 ms |
| auto 4 | 120 ms | 141 ms | 144 ms |
| drl 0 | 118 ms | 139 ms | 140 ms |
| drl 1 | **116 ms** | **137 ms** | **135 ms** |
| drl 2 | 119 ms | 145 ms | 145 ms |
| drl 3 | 116 ms | 139 ms | 143 ms |
| drl 4 | 122 ms | 139 ms | 143 ms |
| fruchterman_reingold 0 | 160 ms | 155 ms | 149 ms |
| fruchterman_reingold 1 | 149 ms | 161 ms | 150 ms |
| fruchterman_reingold 2 | 150 ms | 166 ms | 147 ms |
| fruchterman_reingold 3 | 152 ms | 159 ms | 149 ms |
| fruchterman_reingold 4 | 150 ms | 162 ms | 152 ms |

The following table collects the average number of extended nodes over all experiments. Since the runs without layout algorithms are identical we will only pick one representation from them. 


| Experiment | Dijkstra | A* | Heuristic Search |
| --- | --- | --- | --- |
| Benchmark 4| 3388818 | 3388818 | 3290487 |
| auto 0 | 3218843 | 2839315 | 2900369 |
| auto 1 | 3274321 | 2850940 | 2896794 |
| auto 2 | 3293419 | 2857085 | 2880796 |
| auto 3 | 3274679 | 2874067 | 2931858 |
| auto 4 | 3324951 | 2895926 | 3014088 |
| drl 0 | 3288192 | 2871106 | 2939742 |
| drl 1 | 3301889 | 2852683 | 2902113 |
| drl 2 | 3257531 | 2844847 | 3087309 |
| drl 3 | 3259777 | 2869445 | 2973507 |
| drl 4 | 3367594 | 2867073 | 2949763 |
| fruchterman_reingold 0 | 3243333 | 2847498 | 3171860 |
| fruchterman_reingold 1 | 3293176 | 2856049 | 3185883 |
| fruchterman_reingold 2 | 3349097 | 2871245 | 3097288 |
| fruchterman_reingold 3 | 3353094 | 2875947 | 3148494 |
| fruchterman_reingold 4 | 3310525 | 2877996 | 3150879 |

### Discussion

In the tables above, we can see different behavious for the three algorithms.
For Dijkstra's Algorithm and A*, the layouting algorithms "drl" and "auto" show a clear improvement over the benchmark while "fruchterman_reingold" shows little improvement or even worse times.

Heuristic search, on the other hand, performs worse for the three layout algorithms than it does without. As can be seen in the number of extended nodes, the algorithm does indeed profit from the coordinates as it visits less nodes. However, the time required per node grows. This is likely due to the need to sort the children of each node in order to identify the order in which to visit them. 

### Conclusion
Variant 1 of drl has the lowest times for all three algorithms. We will thus continue with this variant.
We will also note that heuristic search perfromed best without layouting algorithm.


## Experiment 3: Which algorithm is best?

The goal of this experiment is to evaluate which of the three algorithms performs best.

### Experiment Setup

Based on the previous experiment we have selected the coordinate generation algorithm DrL in variation 1.
We will run A*, Dijkstra and Heuristic search, as described in the previous experiment, on these coordinates to determine which algorithm performs best. 

Each run will be comprised of 1000 planning problems, all with the same seed. From each experiment we will collect the following data:
- *average_time*: The average time required took to solve one problem.
- *nr_extended*: The number of nodes that were extended.
- *time per extension*: The average time required for one extension.
We will collect this data for the dataset as a whole but also for only sucessful / failed runs.

While only the overall average time will be used for selecting which algorithm to test further, we will use the other data points for discussion about the differences between algorithm performance.


In [5]:

algorithm_name = "drl"
scale_factor = 10
variation = 1
planners = [Dijkstra, A_Star, Heuristic_Search] 


file_name = f"if_{algorithm_name}_{variation}"
dataset = Dataset()
graph = dataset.load_graph()
dataset.convert_to_spatial(graph,file_name, scale_factor=scale_factor)

for planner in planners:
    print(f"\nrunning {planner} on algorithm {algorithm_name} with variation {variation}")
    experiment = Timed_Experiment(graph, planner, 1000, random_seed=42, verbose = False)
    experiment.run()
    print("Average Time (All): ", int(experiment.get_average_time()), "ns")
    print("Nr Extended (All): ", experiment.get_nr_extensions())
    print("Time per extension (All): ", int(experiment.get_average_extension_time()), "ns")

    print("Successes:", len(experiment.successful_extensions))
    print("Average Time (Success):", int(experiment.get_average_successful_time()) , "ns")
    print("Nr Extended (Success): ", experiment.get_nr_successful_extensions())
    print("Time per extension (Success): ", int(experiment.get_average_successful_extension_time()), "ns")

    print("Fails:", len(experiment.unsuccesful_extensions))
    print("Average Time (Fail):", int(experiment.get_average_unsuccessful_time()) , "ns")
    print("Nr Extended (Fail): ", experiment.get_nr_unsuccessful_extensions())
    print("Time per extension (Fail): ", int(experiment.get_average_unsuccessful_extension_time()), "ns")



running <class 'algorithms.dijkstra.Dijkstra'> on algorithm drl with variation 1
Average Time (All):  137535175 ns
Nr Extended (All):  33701771
Time per extension (All):  4080 ns
Successes: 151
Average Time (Success): 114115022 ns
Nr Extended (Success):  4001968
Time per extension (Success):  4305 ns
Fails: 849
Average Time (Fail): 141700596 ns
Nr Extended (Fail):  29699803
Time per extension (Fail):  4050 ns

running <class 'algorithms.a_star.A_Star'> on algorithm drl with variation 1
Average Time (All):  165718314 ns
Nr Extended (All):  30234781
Time per extension (All):  5481 ns
Successes: 151
Average Time (Success): 31341823 ns
Nr Extended (Success):  534978
Time per extension (Success):  8846 ns
Fails: 849
Average Time (Fail): 189618020 ns
Nr Extended (Fail):  29699803
Time per extension (Fail):  5420 ns

running <class 'algorithms.heuristic_search.Heuristic_Search'> on algorithm drl with variation 1
Average Time (All):  163107504 ns
Nr Extended (All):  30883937
Time per extensio

### Results

| Value | Dijkstra | A* | Heuristic Search |
| --- | --- | --- | --- |
| Average Time (All)| **138 ms** | 166 ms | 163 ms |
| Average Time (Success)| 114 ms | 31 ms  | 44 ms |
| Average Time (Fail)| 142 ms  | 189 ms | 184 ms |
| Nr Extended (All)| 33.701.771 | 30.234.781 | 30.883.937 |
| Nr Extended (Success)| 4.001.968 | 534.978 | 1.184.039 |
| Nr Extended (Fail)| 29.699.803 | 29.699.803 | 29.699.898 |
| Time per extension (All)| 4080 ns | 5481 ns | 5281 ns |
| Time per extension (Success)| 4305 ns | 8846 ns | 5721 |
| Time per extension (Fail)| 4050  | 5420 ns | 5263 |


### Discussion

Overall, the fastest Algorithm on this dataset is Dijkstra's Algorithm.


In addition to this general conclusion, we can derive a few more insights from the experiment:
- In all three algorithms, the average time for sucessful runs were significantly lower than those for failed runs. On the one hand, this can be attributed to the fact that in failed runs the entire connected part of the graph has to be searched while sucessful runs can terminate once they find the goal. On the other hand, A* and Heuristic search perform significantly better that Dijkstra's algorithm on those cases. This indicates that the heuristic is helpful in reducing run time of these algorithms. In particular, A* reduces the number of extended nodes by a factor of almost ten.
- A* is the algorithm that profits the most from the heuristic. The difference in average times between sucessful and failed runs is the by far the biggest of the three. The reason that A* does not come out best overall is likely twofold:
   - A* has the highest time per extension due to the algorithmic complexity of keeping a priority queue of nodes and calculating the heuristci. 
   Due to the overall low number of successful cases (abouut 15% in this experiment), the advantage of A* does not come into play often enough for it to matter in the aveage time.
- Heuristic search is faster mostly due to it's lower per extension time. While it extends more states than A* (though less than Dijkstra's Algorithm), it requires less time per state.

### Conclusions

Dijkstra's algorithm will be used in future experiments.
Since Heuristic Search without coordinate annotation beats Dijkstra's algorithm, we will also include this algorithm in future experiments.
